In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, pinv
%matplotlib inline
np.set_printoptions(precision=12, suppress=True)

In [2]:
consider_distort = False # flag whether to consider distortion
zoomed_coords = True # whether to divide coordinates by SCALE_FAC or not (zoomed coords or not)
center_only = True # use only central stars
SCALE_FAC = 4.0 # Scale factor of coordinates

In [3]:
coords_folder = 'data/star_coords/2016nov-11_txt/' # folder with coords files
images_folder = 'data/stars/2016nov-11/'
results_folder = 'results/2016nov-11/'

In [4]:
# 20160218
# fnames = [
#     '20160218-201934-437.txt',
#     '20160218-211934-484.txt'
# ]

# 20160219
# fnames = [
#     '20160219-001934-484.txt',
#     '20160219-021934-593.txt',
#     '20160219-041934-593.txt'
# ]

# 20160311
# fnames = [
#     '20160311-020811-125.txt',
#     '20160311-030811-125.txt'
# ]

# 2016nov-11 jpg
fnames = [
    "20161122-191517-359.txt",
    "20161121-220921-250.txt",
]

In [5]:
date = fnames[0][:-4]
print(date)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(date)? (<ipython-input-5-554114b46472>, line 2)

In [ ]:
if consider_distort:
    print("consider_distort is True!")

In [ ]:
im = Image.open(images_folder + "mod_" + date + "-1.jpg")
w, h = im.size
print("Image size:", w, h)

In [ ]:
CENTER_RAD = h/4 # radius(px) of central part
x_c, y_c = w/2, h/2 # central point of img
print 'x_c, y_c:', x_c, y_c
print 'CENTER_RAD:', CENTER_RAD

In [ ]:
coords_list = []
for fname in fnames:
    piece = np.loadtxt(coords_folder + os.sep + fname)
    coords_list.append(piece)

coords = np.vstack(coords_list)
print 'Zoomed In Star coordinates pairs:\n', coords, '\n'

if zoomed_coords:
    coords /= float(SCALE_FAC)
    coords = coords.round()
    print 'Normal Star coordinates pairs:\n', coords


if center_only:
    coords_center = []
    
    for i in xrange(coords.shape[0]):
        _lx = coords[i, 0]
        _ly = coords[i, 1]
        _rx = coords[i, 2]
        _ry = coords[i, 3]
        if \
        (_lx - x_c)**2 + (_ly - y_c)**2 <= CENTER_RAD**2 and \
        (_rx - x_c)**2 + (_ry - y_c)**2 <= CENTER_RAD**2:
            print coords[i]
            coords_center.append(coords[i])
    
    coords = np.vstack(coords_center)
    print 'Normal Star coordinates pairs in center:\n', coords

In [ ]:
lX = coords[:, 0] # leftX coordinates
lY = coords[:, 1] # leftY coordinates
rX = coords[:, 2] # rightX coordinates
rY = coords[:, 3] # rightY coordinates

N = coords.shape[0] # number of pairs of points
M = coords.shape[1] # lX, lY, rX, rY == 4
print 'Number of Star coordinates pairs:', N

In [ ]:
xi = np.zeros(2*N)

num_coeffs = 6
if consider_distort:
    num_coeffs = 8

z = np.zeros(num_coeffs)
arr = np.zeros((2*N, num_coeffs)) # matrix A
print 'Number of coefficients:', num_coeffs

In [ ]:
for i in xrange(N): # fill the xi vector
    xi[2*i] = rX[i]
    xi[2*i + 1] = rY[i]

In [ ]:
if consider_distort:
    im = Image.open('data/stars/2016nov-11/mod_'+ date +'-1.jpg')
    x_c = im.width / 2.0
    y_c = im.height / 2.0
    for i in xrange(N): # fill the A matrix
        dist_l = (lX[i]-x_c)**2 + (lY[i]-y_c)**2
        dist_r = (rX[i]-x_c)**2 + (rY[i]-y_c)**2

        zx1 = (lX[i] - x_c) * dist_l
        zx2 = (rX[i] - x_c) * dist_r
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0, -zx1, zx2]

        zy1 = (lY[i] - y_c) * dist_l
        zy2 = (rY[i] - y_c) * dist_r
        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1, -zy1, zy2]

else:
    for i in xrange(N): # fill the A matrix
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0]

        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1]

In [ ]:
np.set_printoptions(precision=2, suppress=True)
print 'A:\n', arr, '\n'
print 'xi:\n', xi
np.set_printoptions(precision=12, suppress=True)

In [ ]:
p_arr = pinv(arr)
z = np.dot(p_arr, xi)
print 'Affine coefficients: \n', z

In [ ]:
sigma = 2 # 2 pix error
I = np.eye(2*N)

print 'cond(A): ', np.linalg.cond(arr)
proba = (2*N - num_coeffs) / norm( np.dot(I - np.dot(arr, p_arr), xi) )**2
sig_theory = (1 / proba)**0.5

print 'Probability:', proba
print 'Sigma theory:', sig_theory
print 'Sigma practice', sigma

In [ ]:
a = float(z[0])
b = float(z[1])
c = float(z[2])
d = float(z[3])
e = float(z[4])
f = float(z[5])
if consider_distort:
    eps1 = float(z[6])
    eps2 = float(z[7])

In [ ]:
if consider_distort:
    # Get rid of distortions on nlImg
    for i in xrange(N): 
        dist_l = (lX[i] - x_c)**2 + (lY[i] - y_c)**2

        zx1 = (lX[i] - x_c) * dist_l
        zy1 = (lY[i] - y_c) * dist_l

#         lX[round(i - eps1*zx1), round(j - eps2*zy1)] = l_pix[i, j]
        lX[i] = lX[i] - eps1 * zx1
        lY[i] = lY[i] - eps1 * zy1
    
    
    # Get rid of distortions on nrImg

    for i in xrange(N): 
        dist_r = (rX[i] - x_c)**2 + (rY[i] - y_c)**2

        zx2 = (rX[i] - x_c) * dist_r
        zy2 = (rY[i] - y_c) * dist_r

#             nr_pix[round(i - eps1*zx2), round(j - eps2*zy2)] = r_pix[i, j]
        rX[i] = rX[i] - eps2 * zx2
        rY[i] = rY[i] - eps2 * zy2


In [ ]:
def affine_transform_point(x, y):
    return [b * y + x * a + e , d * y + x * c + f]

In [ ]:
# Calc new (affine transformed) points
for i in xrange(N): 
    lX[i], lY[i] = affine_transform_point(lX[i], lY[i])

In [ ]:
mse_dist = 0 # MSE of distance
mse_x = 0 # MSE of x
max_d = 0 # Max of distance error

# alignment errors (px) = sqrt( (lX[i] - rX[i])**2 + (lY[i] - rY[i])**2 )
align_errs = np.zeros(N) 
dists = np.zeros(N) # distance from (x_c, y_c) to lX[i], lY[i]

for i in xrange(N):
    mse_dist += (lX[i] - rX[i])**2 + (lY[i] - rY[i])**2
    mse_x += (lX[i] - rX[i])**2
    
    align_errs[i] = np.sqrt((lX[i] - rX[i])**2 + (lY[i] - rY[i])**2)
    dists[i] = np.sqrt((lX[i] - x_c)**2 + (lY[i] - y_c)**2)
    
    if (lX[i] - rX[i])**2 + (lY[i] - rY[i])**2 > max_d:
        max_d = (lX[i] - rX[i])**2 + (lY[i] - rY[i])**2

mse_dist = (1.0 * mse_dist / N)**0.5
mse_x = (1.0 * mse_x / N)**0.5
max_d = max_d**0.5

print 'mse_dist:', mse_dist
print 'mse_x:', mse_x
print 'max_d:', max_d

In [ ]:
res_fname = results_folder

if center_only:
    res_fname += 'center_only_CenRad' + str(CENTER_RAD) +"_"

if consider_distort:
    res_fname += 'affdis_coeffs_shots'
else:
    res_fname += 'affine_coeffs_shots'


res_fname += str(len(fnames)) + '_' + fnames[0]

In [ ]:
np.savetxt(res_fname, z, fmt="%.12f")
print 'Coeffs are saved to:', res_fname

In [ ]:
if center_only:
    plt.figure(figsize=(8,6))
    plt.plot(dists, align_errs, 'o')
    plt.xlabel('Distance from center point to (lX[i], lY[i]), px')
    plt.ylabel('Alignment error distance, px')
    plt.savefig(res_fname[:-4] + '_plot_AlignErr_from_Distance')